In [2]:
# import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta


In [3]:
df = pd.read_csv('AAPL.csv')

df['Date'] = pd.to_datetime(df['Date'])

df.index = df['Date']

df.head()


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-01-22,2015-01-22,110.260002,112.470001,109.720001,112.400002,103.275063,53796400
2015-01-23,2015-01-23,112.300003,113.750000,111.529999,112.980003,103.807991,46464800
2015-01-26,2015-01-26,113.739998,114.360001,112.800003,113.099998,103.918228,55615000
2015-01-27,2015-01-27,112.419998,112.480003,109.029999,109.139999,100.279716,95568700
2015-01-28,2015-01-28,117.629997,118.120003,115.309998,115.309998,105.948822,146477100


In [4]:
df = df.drop(["Date"], axis=1)

In [5]:
#5.1. Step 1: extract technical indicators

#Define Technical Indicators
#_______________________________________________________________________

#Simple Moving Average  
sma = ta.SMA(df.Close,15)
df['MA15'] = sma
df = df.fillna(0)

# RSI
df["RSI"] = ta.RSI(df.Close,14)

# Stochastic K
df["Stoch_K"], df["Stoch_D"] = ta.STOCH(df["High"], df["Low"], df["Close"], fastk_period=14, slowk_period=3, slowk_matype=0, 
                                       slowd_period=3, slowd_matype=0)

# MACD
df['MACD'], df['MACDSignal'], df['MACDHist'] = ta.MACD(df.Close, fastperiod= 12, slowperiod= 26, signalperiod= 9)

# Williams R%
df["WILLR"] = ta.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14) 

#df["WILLR"] = (WILLR - WILLR.min())/(WILLR.max() - WILLR.min())
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA15,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR
Date,,,,,,,,,,,,,,
2020-01-14,316.700012,317.570007,312.170013,312.679993,312.679993,40488600,298.793996,77.037165,92.909134,94.984096,11.280343,9.855563,1.424780,-14.112597
2020-01-15,311.850006,315.500000,309.549988,311.339996,311.339996,30480900,300.616663,74.570538,88.878048,92.854598,11.131528,10.110756,1.020772,-18.953489
2020-01-16,313.589996,315.700012,312.089996,315.239990,315.239990,27207300,302.681329,76.889809,86.577130,89.454770,11.199191,10.328443,0.870748,-7.202524
2020-01-17,316.269989,318.739990,315.000000,318.730011,318.730011,34454100,304.602663,78.756961,91.271405,88.908861,11.402984,10.543351,0.859633,-0.029770
2020-01-21,317.190002,319.019989,316.000000,316.570007,316.570007,27664200,306.387331,74.732596,94.820894,90.889810,11.260394,10.686760,0.573634,-8.305024


In [6]:
# 5.2. Step 2: trend analysis using technical indicators

#make MA15 into a series
MA15 = df.iloc[:,6]

# Up trend is 1, No trend is 0, and Down trend is -1
df['Trend'] = MA15.rolling('5d').apply(lambda x: np.sign(x[-1] - x[0]))

df.tail()
#If close > MA15 and MA15 is rising for last 5 days then 1 (Uptrend)
#If close < MA15 and MA15 is declining for last 5 days then -1 (Downtrend)
#If neither are true then 0 (Notrend)
def f(row):
    if row['Close'] > row['MA15'] and row['Trend'] == 1:
        val = 1
    elif row['Close'] < row['MA15'] and row['Trend'] == -1:
        val = -1
    else:
        val = 0
    return val
df['Trend'] = df.apply(f, axis=1)
df.tail()

C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


,Open,High,Low,Close,Adj Close,Volume,MA15,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend
Date,,,,,,,,,,,,,,,
2020-01-14,316.700012,317.570007,312.170013,312.679993,312.679993,40488600,298.793996,77.037165,92.909134,94.984096,11.280343,9.855563,1.424780,-14.112597,1
2020-01-15,311.850006,315.500000,309.549988,311.339996,311.339996,30480900,300.616663,74.570538,88.878048,92.854598,11.131528,10.110756,1.020772,-18.953489,1
2020-01-16,313.589996,315.700012,312.089996,315.239990,315.239990,27207300,302.681329,76.889809,86.577130,89.454770,11.199191,10.328443,0.870748,-7.202524,1
2020-01-17,316.269989,318.739990,315.000000,318.730011,318.730011,34454100,304.602663,78.756961,91.271405,88.908861,11.402984,10.543351,0.859633,-0.029770,1
2020-01-21,317.190002,319.019989,316.000000,316.570007,316.570007,27664200,306.387331,74.732596,94.820894,90.889810,11.260394,10.686760,0.573634,-8.305024,1


In [7]:
#I'm not sure if this is right, I would appreciate it if you double checked my work here

#5.3. Step 3: trading signal generation from trend analysis

Close = df.iloc[:,3]
df["UpTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5 + .5
df["DownTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5

def g(row):
    if row['Trend'] == 1:
        val = row['UpTrend']
    elif row['Trend'] == -1:
        val = row['DownTrend']
    else:
        val = 0
    return val
df['Trade_Signal'] = df.apply(g, axis=1)
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA15,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend,UpTrend,DownTrend,Trade_Signal
Date,,,,,,,,,,,,,,,,,,
2020-01-14,316.700012,317.570007,312.170013,312.679993,312.679993,40488600,298.793996,77.037165,92.909134,94.984096,11.280343,9.855563,1.424780,-14.112597,1,0.5,0.0,0.5
2020-01-15,311.850006,315.500000,309.549988,311.339996,311.339996,30480900,300.616663,74.570538,88.878048,92.854598,11.131528,10.110756,1.020772,-18.953489,1,0.5,0.0,0.5
2020-01-16,313.589996,315.700012,312.089996,315.239990,315.239990,27207300,302.681329,76.889809,86.577130,89.454770,11.199191,10.328443,0.870748,-7.202524,1,1.0,0.5,1.0
2020-01-17,316.269989,318.739990,315.000000,318.730011,318.730011,34454100,304.602663,78.756961,91.271405,88.908861,11.402984,10.543351,0.859633,-0.029770,1,1.0,0.5,1.0
2020-01-21,317.190002,319.019989,316.000000,316.570007,316.570007,27664200,306.387331,74.732596,94.820894,90.889810,11.260394,10.686760,0.573634,-8.305024,1,NaN,NaN,NaN


In [41]:
# df.count

In [43]:
#What do I use for the target Column? and should I fill null values with 0?

#I added .tail(n=Length - 17) to omit values where the indicators = 0 since the machine learning agorithm trains on the values 
#of the indicator values and the Stock_K/D indicators require 17 previous close prices to calculate.
Length = df.shape[0]

# df2 = (df[['MA15', 'RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trade_Signal']].tail(n=1242)).copy()

df2 = (df[['MA15', 'RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trade_Signal']].tail(n=Length - 17)).copy()

df2 = df2.fillna(0)

df2.describe()

,MA15,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
count,1241.000000,1241.000000,1241.000000,1241.000000,1241.000000,1241.000000,1241.000000
mean,155.991965,55.746946,60.379380,60.382478,0.942485,-39.622439,0.387913
std,45.063842,13.827458,28.957516,28.218103,3.254679,30.885622,0.412816
min,93.227999,21.024931,0.586542,3.392294,-11.052513,-100.000000,0.000000
25%,116.116667,45.811844,35.087663,34.640090,-0.575418,-67.741853,0.000000
50%,152.930000,56.539717,67.919159,67.820383,1.010995,-32.336559,0.330084
75%,187.194667,65.304248,86.496496,85.993819,2.754122,-11.471061,0.851351
max,306.387331,90.402156,99.149279,98.329441,11.402984,-0.000000,1.000000


In [44]:
#5.5. Step 5: network structure creation and training using ELM

#This is where I need the most help, I'm  not sure if I'm using the all the right modules and functions
# and I need the output to fill a column with the output values 0 through 1 so I can move to-
# Step 6: trend determination from output trading signal

target_column = ['Trade_Signal'] 
predictors = list(set(list(df2.columns))-set(target_column))
#line 3 here fufills the data normalization of Step 5.4
df2[predictors] = (df2[predictors] - df2[predictors].min())/(df2[predictors].max() - df2[predictors].min())

#df2.describe()
#df2.info()
# df2.head()


df2

,MA15,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
Date,,,,,,,
2015-02-17,0.126913,0.807767,0.956434,0.982306,0.492196,0.922623,0.000000
2015-02-18,0.133037,0.820761,0.971068,0.977231,0.492196,0.987988,1.000000
2015-02-19,0.137146,0.806512,0.963253,0.970830,0.492196,0.955212,0.848312
2015-02-20,0.140461,0.823244,0.989422,0.982246,0.492196,1.000000,1.000000
2015-02-23,0.145415,0.871538,0.993484,0.990003,0.492196,1.000000,0.000000
2015-02-24,0.149650,0.827294,0.977001,0.994761,0.492196,0.906474,0.500000
2015-02-25,0.152822,0.667042,0.869769,0.953354,0.492196,0.682926,0.500000
2015-02-26,0.156218,0.699364,0.798875,0.886007,0.492196,0.790375,0.741125
2015-02-27,0.158883,0.617604,0.715916,0.795654,0.492196,0.661174,0.500000


In [28]:
X = df2[predictors]

y = df2[target_column]

# X = X.T

print(X.shape) ; print(y.shape)

(1242, 6)
(1242, 1)


In [29]:
Xindex = np.array([[1], [2], [3], [4], [5], [6]])

Xindex = Xindex.T

print(Xindex.shape); print(Xindex)

(1, 6)
[[1 2 3 4 5 6]]


In [33]:
#  FEB

# cx = np.outer(X, Xindex).flatten(order='F')

# cx = np.sum(cx.reshape(-1, 6), axis=1)

# cx = np.tanh(cx)

#all on one line
cx = np.tanh(np.sum(np.outer(X, Xindex).flatten(order='F').reshape(-1, 6), axis=1))



print(cx.shape); print(cx)


(7452,)
[0.98333642 0.99962375 0.9996883  ... 1.         1.         1.        ]


In [34]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)
# print(cx)

In [35]:
synaptic_weights = np.random.random(len(cx))
print(synaptic_weights)

[0.24505985 0.34098946 0.73086298 ... 0.92356024 0.78089513 0.63185647]


In [ ]:
# for iteration in range(1000):
#     z = np.dot(cx, synaptic_weights)
#     sigmoid = 1/(1+np.exp(-z))
#     error = (y - sigmoid)
# # print(error); print(cx.shape)
# print(error.shape)

# for iteration in range(1000):
#     z = np.dot(cx, synaptic_weights)
#     sigmoid = 1/(1+np.exp(-z))
#     error = (y - sigmoid)
# print(error); print(cx.shape)
# print(error.shape)

In [ ]:
# sigmoidDerivative = sigmoid * (1 - sigmoid)
# # print(synaptic_weights.shape)
# # print(sigmoidDerivative)

In [ ]:
# synaptic_weights += np.dot(y, error*sigmoidDerivative)

In [ ]:
# print("Considering new situation: [0,1,1]")
# # calculate weighted inputs
# newZ = np.dot(np.array([0,1,1,1,1,1]), synaptic_weights)
# # put weighted inputs into our activation function to get the network’s output
# activationOutput = 1/(1+np.exp(-newZ))
# print(activationOutput.shape)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
# print(X_train.shape); print(X_test.shape)

In [ ]:
# # not having this causes model.fit(X_train, y_train, batch_size=500) to have an accuracy of 1 which doesn't seem right.

# # one hot encode outputs
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# count_classes = y_test.shape[1]
# print(count_classes)